In [1]:
import PySpin
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import datetime
import csv

In [2]:
def configure_camera(camera):
    # Changes amount of frame captured in camera
    width = int(2448 / 1)
    height = int(2048 / 1)

    # Reset Camera
    nodemap = camera.GetNodeMap()
    user_set_selector = PySpin.CEnumerationPtr(nodemap.GetNode("UserSetSelector"))
    default_set = user_set_selector.GetEntryByName("Default")
    user_set_selector.SetIntValue(default_set.GetValue())
    user_set_load = PySpin.CCommandPtr(nodemap.GetNode("UserSetLoad"))
    user_set_load.Execute()
    
    if camera.PixelFormat.GetAccessMode() == PySpin.RW:
        camera.PixelFormat.SetValue(PySpin.PixelFormat_RGB8)
        print("Pixel format set to color.")
    else:
        print("Color pixel format not supported.")

    # Retrieve the nodemap
    nodemap = camera.GetNodeMap()

    # Get width and height nodes
    node_width = PySpin.CIntegerPtr(nodemap.GetNode("Width"))
    node_height = PySpin.CIntegerPtr(nodemap.GetNode("Height"))
    node_width_max = node_width.GetMax()
    node_height_max = node_height.GetMax()

    # Calculate offsets to center the image
    offset_x = (node_width_max - width) // 2
    offset_y = (node_height_max - height) // 2

    # Set width and height
    node_offset_x = PySpin.CIntegerPtr(nodemap.GetNode("OffsetX"))
    node_offset_y = PySpin.CIntegerPtr(nodemap.GetNode("OffsetY"))

    node_width.SetValue(width)
    node_height.SetValue(height)
    node_offset_x.SetValue(offset_x)
    node_offset_y.SetValue(offset_y)

    camera.AcquisitionMode.SetValue(PySpin.AcquisitionMode_SingleFrame)

In [12]:
def capture_and_process_image(camera, model, video_writer, csv_writer):
    camera.BeginAcquisition()
    image_result = camera.GetNextImage()
    if image_result.IsIncomplete():
        print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
        image_result.Release()
        camera.EndAcquisition()
        return True

    image_data = image_result.GetNDArray()
    image_result.Release()
    camera.EndAcquisition()
    
    # Convert to a format that can be displayed with OpenCV
    image_data = cv2.resize(image_data, (640, 640))
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    # Perform detection
    results = model.track(image_data, classes=0, agnostic_nms=False, verbose=False, persist = True)[0]
    annotator = Annotator(image_data)
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for r in results:
        for box in r.boxes:
            print(box)
            annotator.box_label(box.xyxy[0], model.names[int(box.cls)])
            csv_writer.writerow([timestamp, *box.xyxy[0], box.conf[0], model.names[int(box.cls)]])

    # Save the frame to video
    video_writer.write(cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR))

    # Display the annotated image
    annotated_image = annotator.result()
    cv2.imshow('Processed Image', annotated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        return False

    return True



In [13]:
def main():
    system = PySpin.System.GetInstance()
    cam_list = system.GetCameras()
    if cam_list.GetSize() == 0:
        print('No cameras found.')
        return

    camera = cam_list.GetByIndex(0)
    camera.Init()
    configure_camera(camera)

    model = YOLO("yolov8n.pt")  # Load your YOLO model

    # Setup video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (640, 640))

    # Setup CSV writer
    csv_file = open('detections.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Timestamp', 'X1', 'Y1', 'X2', 'Y2', 'Confidence', 'Class'])

    while True:
        if not capture_and_process_image(camera, model, video_writer, csv_writer):
            break

    camera.DeInit()
    del camera
    cam_list.Clear()
    system.ReleaseInstance()
    cv2.destroyAllWindows()
    video_writer.release()
    csv_file.close()

In [14]:
if __name__ == '__main__':
    main()

Pixel format set to color.
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.5595])
data: tensor([[3.4380e+01, 1.0800e+01, 4.8936e+02, 6.3089e+02, 5.5948e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[261.8710, 320.8430, 454.9829, 620.0861]])
xywhn: tensor([[0.4092, 0.5013, 0.7109, 0.9689]])
xyxy: tensor([[ 34.3795,  10.8000, 489.3624, 630.8860]])
xyxyn: tensor([[0.0537, 0.0169, 0.7646, 0.9858]])
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.4941])
data: tensor([[3.6412e+00, 3.1714e+00, 6.1655e+02, 6.3796e+02, 4.9414e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[310.0939, 320.5652, 612.9054, 634.7875]])
xywhn: tensor([[0.4845, 0.5009, 0.9577, 0.9919]])
xyxy: tensor([[  3.6412,   3.1714, 616.5466, 637.9589]])
xyxyn: tensor([[0.0057, 0.0050, 0.9634, 0.9968]])
ultralytics.engine.re